
# Ferramenta de Análise Astronômica (Jupyter Notebook)
...


In [ ]:

import warnings
from datetime import date
import pytz
from src.config import *
from src.location import get_location_from_city, set_timezone_for_sao_paulo
from src.targets import get_target_skycoords, registrar_alvos_sistema_solar, DEEP_SKY_TARGETS_PRESET
from src.analysis import calculate_nightly_events, analyze_target_visibility_for_night, analyze_year_visibility
from src.plotting import plot_target_visibility, plot_yearly_visibility
warnings.filterwarnings('ignore', category=AstropyWarning)
warnings.filterwarnings('ignore', category=erfa.ErfaWarning)
print("Módulos e dependências carregados com sucesso.")



---
## ⚙️ 2. Configurações da Análise
**Edite as variáveis nesta célula para personalizar sua análise.**


In [ ]:

from datetime import date
NOME_DA_CIDADE = "Vitória da Conquista, Brazil"
DATA_ANALISE = date(2024, 7, 15)
ELEVACAO_MINIMA_GRAUS = 30
usar_alvos_predefinidos = True
usar_alvos_sistema_solar = True
alvos_manuais = ["NGC 5128", "M83"]
ALVO_ANUAL = "M42"
ANO_ANALISE = 2024



---
## 🌙 3. Execução da Análise Noturna
...


In [ ]:

observer_location = get_location_from_city(NOME_DA_CIDADE)
observer_timezone = set_timezone_for_sao_paulo(observer_location) or pytz.UTC
if observer_location is not None:
    print(f"Análise para {NOME_DA_CIDADE} na data {DATA_ANALISE.strftime('%d/%m/%Y')}")
    night_events = calculate_nightly_events(DATA_ANALISE, observer_location, observer_timezone)
    start_night, end_night = night_events['inicio_noite'], night_events['fim_noite']
    print(f"Janela de observação: de {start_night.to_datetime(observer_timezone).strftime('%H:%M')} a {end_night.to_datetime(observer_timezone).strftime('%H:%M')}")
    
    nomes_alvos = []
    if usar_alvos_predefinidos: nomes_alvos.extend(DEEP_SKY_TARGETS_PRESET)
    if alvos_manuais: nomes_alvos.extend(alvos_manuais)
    
    all_targets = {}
    if nomes_alvos: all_targets.update(get_target_skycoords(nomes_alvos))
    if usar_alvos_sistema_solar: all_targets.update(registrar_alvos_sistema_solar(start_night))
    
    for name, coord in all_targets.items():
        if coord is not None:
            df_visibility = analyze_target_visibility_for_night(start_night, end_night, observer_location, coord, ELEVACAO_MINIMA_GRAUS * u.deg)
            fig = plot_target_visibility(df_visibility, name, DATA_ANALISE, ELEVACAO_MINIMA_GRAUS)
            plt.show()
        else:
            print(f"Não foi possível obter coordenadas para {name}.")



---
## 📅 4. Execução da Análise Anual
...


In [ ]:

if observer_location is not None and ALVO_ANUAL:
    print(f"Gerando calendário anual para '{ALVO_ANUAL}' em {ANO_ANALISE}...")
    target_coords_dict = get_target_skycoords([ALVO_ANUAL])
    target_coord = target_coords_dict.get(ALVO_ANUAL)
    if target_coord is not None:
        df_year = analyze_year_visibility(ANO_ANALISE, observer_location, observer_timezone, target_coord, ELEVACAO_MINIMA_GRAUS * u.deg)
        if not df_year.empty:
            fig = plot_yearly_visibility(df_year, ALVO_ANUAL, ANO_ANALISE)
            plt.show()
        else:
            print("Nenhum período de visibilidade encontrado.")
    else:
        print(f"Não foi possível encontrar o alvo '{ALVO_ANUAL}'.")
